In [1]:
import pandas as pd
import numpy as np
import pyomo.environ as pyo
from numpy.random import normal

In [2]:
#  weight parameters are uncertain with normal distribution and known mean, standard deviation
df = pd.read_csv('knapsack_dataset_uncertain_weight.csv')  
print(df)

     name  value  weight_mean  weight_std
0    gold    100          4.0         0.1
1  silver     90          2.8         0.4
2  bronze     70          2.2         0.3


In [3]:
knapsack_capacity = 6

In [25]:
%%latex 
Binary Knapsack problem with uncertainty in the weight of items. In this problem we assume that the weight of items are known
only after we made our decisions.
$$\max \sum\limits_{s\in S}p_s\sum\limits_{i = 1}^n v_i x_i\;\;\;\;[is\,equivalent\; to: ]\max \sum\limits_{i = 1}^n v_i x_i$$

$$\sum\limits_{i = 1}^n  w^s_i x_i \leq b,\; \forall s \in S$$
$$x_i\in \{0, 1\}\;\forall i=1,\;...,n$$

<IPython.core.display.Latex object>

In [5]:
# generating scenarios:
number_of_scenarios = 128
scenarios = normal(df['weight_mean'], df['weight_std'], [number_of_scenarios, len(df)])
# print(scenarios)

In [6]:
# print average of each parameter value over all scenarios
print([np.average(scenarios[:,0]),np.average(scenarios[:,1]),np.average(scenarios[:,2])])

[3.971166202889311, 2.758793577543864, 2.162014750476906]


In [7]:
def init_weight_param(model, i, j):
    return scenarios[i, j]

In [8]:
def init_value_param(model, i):
    return df.loc[i, 'value']

In [9]:
def weight_limit_rule(m, j):
    return sum(m.w[j, i] * m.x[i] for i in range(len(df))) <= knapsack_capacity

In [10]:
model = pyo.ConcreteModel()

In [11]:
# indices definition
model.ind_items = pyo.RangeSet(0, len(df)-1)
model.ind_scenarios = pyo.RangeSet(0, number_of_scenarios-1)

In [12]:
# variable definition
model.x = pyo.Var(model.ind_items, domain=pyo.Binary)

In [13]:
# parameter defnition
model.w = pyo.Param(model.ind_scenarios, model.ind_items,
                    initialize=init_weight_param, within=pyo.PositiveReals)
model.v = pyo.Param(model.ind_items, initialize=init_value_param, within=pyo.PositiveReals)

In [14]:
# objective function definition
model.obj = pyo.Objective(expr = sum((1/number_of_scenarios) * sum(model.v[i] * model.x[i] for i in model.ind_items)
                          for _ in model.ind_scenarios), sense=pyo.maximize)

In [15]:
# constraints definition
model.weight_limit_constraints = pyo.Constraint(model.ind_scenarios, rule=weight_limit_rule)

In [16]:
# solving the model
opt = pyo.SolverFactory('gurobi')
result = opt.solve(model)

In [17]:
model.display()

Model unknown

  Variables:
    x : Size=3, Index=ind_items
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   1.0 :     1 : False : False : Binary
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 100.0

  Constraints:
    weight_limit_constraints : Size=128
        Key : Lower : Body               : Upper
          0 :  None :  4.133317731266521 :   6.0
          1 :  None : 3.9520823807499825 :   6.0
          2 :  None : 3.9418033789227196 :   6.0
          3 :  None :  4.000016304786685 :   6.0
          4 :  None : 3.9703557934986775 :   6.0
          5 :  None : 3.9608421091655055 :   6.0
          6 :  None :   4.20110436996213 :   6.0
          7 :  None :  4.084039122223338 :   6.0
          8 :  None : 3.9269704498821616 :   6.0
          9 :  None :  3.74045454